# imports:

In [2]:
from math import pi, sin, cos
from typing import Sequence
import numpy as np
import plotly.graph_objects as go
import random

# Classes

In [3]:
class Object:
            """
            the class to create and handle fractal Objects

            methods:

            shift:
                the function to apply the shift on the object
            scale:
                the function to apply the scaling on the object
            rotate:
                the function to apply the rotation on the object
            copy:
                creates a new object with the same values for attributes
            """
            def __init__(self,length:float,startingPoint:Sequence[float],theta:float) -> None:
                """
                creates objects based on the params

                :param length: length of the object
                :type length: float
                :param startingPoint:the numpy vector pointing to the starting point of the object and stored as a np.array
                :type startingPoint: Sequence[float]
                :param theta: the orientation of the object
                :type theta: float
                """
                self.length = np.array(length)
                self.startingPoint = np.array(startingPoint)
                self.theta = np.array(theta)

            def shift(self, shiftParams:np.array) -> object:
                """
                the function to apply the shift on the object
                R is the rotating matrix witch should be applied on the shift vector

                :param shiftParams:(the shift vector /length of object)
                :type shiftParams:np.array
                :return:the exact object
                :rtype:object
                """
                R = np.array([[cos(self.theta),-sin(self.theta)],[sin(self.theta),cos(self.theta)]])
                self.startingPoint += R.dot(self.length*shiftParams)
                return self

            def scale(self, r: float) -> object:
                """
                the function to apply the scaling on the object

                :param r:the scaling coefficient
                :type r: float
                :return:the exact object
                :rtype:object
                """
                self.length *= r
                return self

            def rotate(self, rotatingParam: float) -> object:
                """
                the function to apply the rotation on the object

                :param rotatingParam: the change of theta
                :type rotatingParam: float
                :return:the exact object
                :rtype:object
                """
                self.theta += rotatingParam
                return self

            def copy(self) -> object:
                """
                creates a new object with the same values for attributes
                :return: the new object
                :rtype: object
                """
                return Object(self.length.copy(),self.startingPoint.copy(),self.theta.copy())

            def getCoordination(self) -> Sequence[Sequence[float]] :
                """
                the function to get coordination of the triangle object

                :return: the coordination
                :rtype:Sequence[Sequence[float]
                """
                x = [self.startingPoint[0],self.startingPoint[0]+self.length/2,self.startingPoint[0]-self.length/2]
                y = [self.startingPoint[1],self.startingPoint[1]-self.length*sin(pi/3),self.startingPoint[1]-self.length*sin(pi/3)]
                return x,y

            def getPolygon(self) -> go.Scatter:
                """
                the function to get the polygon object based on the coordination of the object

                :return:the polygon(in this case the triangle)
                :rtype:go.scatter
                """
                x,y = self.getCoordination()
                polygon = go.Scatter(
                x=x,
                y=y,
                showlegend=False,
                mode="lines",
                fill="toself",
                line=dict(color="LightSeaGreen", width=2),)
                return polygon


In [4]:
class Transformer:
    """
    the class to create and perform transformers

    methods:
    subTransform:
        to apply all changes of a transformer on an object and return aa new object
    """
    def __init__(self, scaleParam: float, shiftParams: np.array, rotateParam: float) -> None:
        """
        creates transformers based on the params

        :param scaleParam:the scale coefficient of transformer
        :type scaleParam: float
        :param shiftParams:the shift vector of the transformer
        :type shiftParams: np.array
        :param rotateParam:the change of theta witch the transformer applies
        :type rotateParam: float
        """
        self.scaleParam =scaleParam
        self.shiftParams = shiftParams
        self.rotateParam = rotateParam

    def subTransform(self, obj: Object) -> Object:
        """
        to apply all changes of a transformer on an object and return aa new object
        the sort of the changes is important.

        :param obj:the first object
        :type obj: Object
        :return: new object
        :rtype: Object
        """
        return obj.copy().scale(self.scaleParam).shift(self.shiftParams).rotate(self.rotateParam)

In [13]:
class Sierpinski:
    """
    the class to create Sierpinski object and simulate the fractal growth

    methods:
        visualize:
            the function to create a plotly plot of the objects at the step
        transform:
            the function to apply all the transformers on the input object
        step:
            the function to go one step forward
        getSequence:
            returns a random sequence of transformers with the input size

        drawRandomFractal
            draws a random Sierpinski triangle fractal plot

    """

    def __init__(self,firstObject,transformers:np.array):
        self.objects:np.array = [firstObject]
        self.transformers = transformers


    def visualize(self) -> None:
        """
        the function to create a plotly plot of the objects at the step
        """
        polygons = []

        for obj in self.objects:
            polygons.append(obj.getPolygon())

        fig = go.Figure(polygons)
        fig.show()

    def transform(self, obj: Object) -> Sequence[Object]:
        """
        the function to apply all the transformers on the input object

        :param obj:the input object
        :type obj:Object
        :return:the new output Objects
        :rtype:Sequence[Object]
        """
        outputs = []
        for transformer in self.transformers:
            outputs.append(transformer.subTransform(obj))

        return outputs

    def step(self) -> None:
        """
        the function to go one step forward
        """
        new_objects = []
        for obj in self.objects:
            new_objects += self.transform(obj)

        self.objects = new_objects

    def getSequence(self, depth: int) -> Sequence[Transformer]:
        """
        returns a random sequence of transformers with the input size

        :param depth:the depth of fractal growth
        :type depth:int
        :return:the random list of transformers with the size of depth
        :rtype:Sequence[Transformer]
        """
        return random.choices(self.transformers,k = depth)


    def drawRandomFractal(self, depth: int, itrations: int) -> None:
        """
        draws a random Sierpinski triangle fractal plot

        :param depth:the depth of fractal growth
        :type depth:int
        :param itrations:the number of redrawing iterations
        :type itrations:int

        """
        polygons = []
        for i in range(itrations):
            sequence = self.getSequence(depth)
            newObj = firstObject
            for i in range(depth):
                newObj = sequence[i].subTransform(newObj)

            polygons.append(newObj.getPolygon())


        fig = go.Figure(polygons)
        fig.show()



###  creating the first triangle and the sierpinski object

In [14]:
firstObject = Object(1.,np.array([0.,1.]),0.)
transformer1 = Transformer(1/2,[0.,0.],0.)
transformer2 = Transformer(1/2,np.array([1/2,-sin(pi/3)]),0.)
transformer3 = Transformer(1/2,np.array([-1/2,-sin(pi/3)]),0.)
sierpinski = Sierpinski(firstObject,[transformer1,transformer2,transformer3])

### drawing the random sierpinski triangle fractal

In [16]:
sierpinski.drawRandomFractal(8,1000)